# benchmark of python caching

In [1]:
from timeit import timeit

In [2]:
def fibonacci(n):
    a,b = 0,1
    for i in range(n):
        a,b = b, a+b
    return a

In [3]:
from functools import lru_cache

@lru_cache(maxsize=501)
def fibonacci_cached(n):
    if n <= 2:
        return 1
    else:
        return fibonacci_cached(n-1)+ fibonacci_cached(n-2)    

## testing a single run

In [4]:
print("fibonacci took:")
print(timeit("fibonacci(500)", number=1, setup="from __main__ import fibonacci"), "s")

fibonacci took:
3.433412281737379e-05 s


In [5]:
print("fibonacci_cached took:")
print(timeit("fibonacci_cached(500)", number=1, setup="from __main__ import fibonacci_cached"), "s")

fibonacci_cached took:
0.0006056659735591101 s


#### making sure that the cache values of different tests do not influence eachother

In [6]:
fibonacci_cached.cache_clear()
fibonacci_cached.cache_info()

CacheInfo(hits=0, misses=0, maxsize=501, currsize=0)

## testsing the creation of all fibonacci series values from the 1st to the 500th

In [7]:
print("fib n=[1:500] fibonacci took:")
print(timeit("[fibonacci(i) for i in range(1, 501)]", number=1, setup="from __main__ import fibonacci"), "s")

fib n=[1:500] fibonacci took:
0.009524707316661808 s


In [8]:
print("fib n=[1:500] fibonacci_cached took:")
print(timeit("[fibonacci_cached(i) for i in range(1, 501)]", number=1, setup="from __main__ import fibonacci_cached"), "s")

fib n=[1:500] fibonacci_cached took:
0.00037014593809256424 s


#### making sure that the cache values of different tests do not influence eachother

In [9]:
fibonacci_cached.cache_clear()
fibonacci_cached.cache_info()

CacheInfo(hits=0, misses=0, maxsize=501, currsize=0)

## jupytermagic shows a "false" value for '%timeit' magic, since it hugely profits from the cache, due to multiple loops

In [10]:
%timeit fibonacci(500)

32.7 µs ± 399 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [11]:
%timeit fibonacci_cached(500)

119 ns ± 0.647 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)


In [12]:
fibonacci_cached.cache_info()

CacheInfo(hits=81111607, misses=500, maxsize=501, currsize=500)

## here 'fibonacci_cached' works since it uses all the cached values

In [13]:
fibonacci_cached(1000)

43466557686937456435688527675040625802564660517371780402481729089536555417949051890403879840079255169295922593080322634775209689623239873322471161642996440906533187938298969649928516003704476137795166849228875

In [14]:
fibonacci_cached.cache_clear()
fibonacci_cached.cache_info()

CacheInfo(hits=0, misses=0, maxsize=501, currsize=0)

## but when you clear the cache it breaks

In [15]:
fibonacci_cached.cache_info()

CacheInfo(hits=0, misses=0, maxsize=501, currsize=0)

In [16]:
fibonacci_cached(1000)

RecursionError: maximum recursion depth exceeded in comparison